# ARIMA

## Decomposition

- components:
 - Trend
 - Seaonal
 - cyclical 

- choose:
 - additive or multiplicative decomposition?   plot the residuls of both models and decide  

- statsmodel.tsa [Deterministic Processes](https://www.statsmodels.org/stable/tsa.html#)      

## if non-stationary convert to stationary TS 

- ADF test
- 1st/2nd order diff or log transform
- plot and ADF test

## hyperparameters

In [ ]:
# plot ACF and PACF

In [ ]:
# grid search the hyper params
import pmdarima as pm
smodel = pm.auto_arima(df.linearized,
                       start_p=0, max_p=2,
                       start_q=0, max_q=2,
                       seasonal=False,
                       trace=True) # The lower AIC the better

## ARIMA Model

In [ ]:
# train
from statsmodels.tsa.arima_model import ARIMA #statsmodels 0.11
#from statsmodels.tsa.arima.model import ARIMA #statsmodels 0.12

arima = ARIMA(df.linearized, order=(2,1,1))
arima = arima.fit()
arima.summary() 

In [ ]:
# predict
arima.plot_predict(end=250);
fig = plt.gcf(); fig.set_size_inches(12, 5);
# MA(1) effect only survives for one time step, no forecast errors after one forecast

In [ ]:
# Create a correct Training/Test split to predict the last 50 points
train = df.linearized[0:150]
test = df.linearized[150:]

# Build Model
arima = ARIMA(train, order=(0, 1, 1))  
arima = arima.fit()

# Forecast
forecast, std_err, confidence_int = arima.forecast(len(test), alpha=0.05)  # 95% confidence
plot_forecast(forecast, train, test, confidence_int[:,0], confidence_int[:,1])

## Recomposition

In [ ]:
# re-compose back to initial time series

forecast_recons = np.exp(forecast)*result_mul.seasonal[150:]
train_recons = np.exp(train)*result_mul.seasonal[0:150]
test_recons = np.exp(test)*result_mul.seasonal[150:]
lower_recons = np.exp(confidence_int)[:,0]*result_mul.seasonal[150:]
upper_recons = np.exp(confidence_int)[:,1]*result_mul.seasonal[150:]

# plt 
plot_forecast(forecast_recons, train_recons, test_recons, lower_recons.values, upper_recons.values)

## Inference validity

In [ ]:
## plot residuals to see whether their normal distributed
residuals = pd.DataFrame(arima.resid)

fig, ax = plt.subplots(1,2, figsize=(16,3))
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

# SARIMA

# Prophet

[reference for model_b](https://towardsdatascience.com/multivariate-time-series-forecasting-using-random-forest-2372f3ecbad1)